In [ ]:
data = np.concatenate( [ dfs[ _kex ].iloc[:,5:].values[:num_rows] for _kex in KEX ], axis = 1 )


### PCA

In [ ]:
_tmp = np.mean( data, axis = 1 )

np.std( data, axis = 1 ) - \
np.sqrt( np.sum( ( data - _tmp[:, np.newaxis] )**2, axis = 1 ) / ( data.shape[1] ) )

In [ ]:
# === PCA

def standardize( data: np.ndarray, mean: np.ndarray, std: np.ndarray ):
  assert ( ( data.ndim == 2 ) and ( mean.ndim == 1 ) and ( std.ndim == 1 ) )
  assert ( data.shape[0] == mean.shape[0] )
  assert ( data.shape[0] == std.shape[0] )
  
  return ( data - mean[:, np.newaxis] ) / std[:, np.newaxis]


def compute_cov( data_stand: np.ndarray ):
  assert ( data_stand.ndim == 2 )
  assert np.all( ( -.0001 < np.mean( data_stand, axis = 1 ) ) & ( np.mean( data_stand, axis = 1 ) < .0001 ) )
  assert np.all( ( .9999 < np.std( data_stand, axis = 1 ) ) & ( np.std( data_stand, axis = 1 ) < 1.0001 ) )

  N = data_stand.shape[1]
  n_feat = data_stand.shape[0]
  cov = ( data_stand @ data_stand.T ) / N
  
  assert ( cov.shape == ( n_feat, n_feat, ) )
  
  return cov

# ------------------- #

mean = np.mean( data, axis = 1 )
std = np.std( data, axis = 1 )

data_stand = standardize( data, mean, std )
cov = compute_cov( data_stand )

# lda, pcomps = np.linalg.eig( cov )
lda, pcomps = eig_scipy( cov )

In [ ]:
np.fabs( np.imag( lda ) ).max()

In [ ]:
lda_argdesc = np.argsort( np.absolute( lda ) )[::-1]

lda_ord = np.absolute( lda[ lda_argdesc ] )
exp_var = lda_ord / np.sum( lda_ord )

np.set_printoptions( suppress = True )
print( np.sum( lda_ord ) )
print( lda_ord[:40] )
print( np.sum( exp_var ) )
print( exp_var[:40] )
print( ((lda_ord[0]+lda_ord[1])/np.sum( lda_ord ))**.5 )
np.set_printoptions( suppress = False )
print()
print( np.sum( lda_ord ) )
print( lda_ord[:40] )
print( np.sum( exp_var ) )
print( exp_var[:40] )
print( ((lda_ord[0]+lda_ord[1])/np.sum( lda_ord ))**.5 )

In [ ]:
np.fabs( np.imag( pcomps[ lda_argdesc ] )[:,:38] ).max(), np.fabs( np.imag( pcomps[ lda_argdesc ] )[:,:39] ).max()

In [ ]:
pcomps_ord = np.absolute( pcomps[ :, lda_argdesc ] )

# All of the principal components need to have magnitude 1 else the projection won't make sense
assert ( ( .9999 < np.all( np.sqrt( np.sum( pcomps_ord**2, axis = 0 ) ) ) ) & \
         ( np.all( np.sqrt( np.sum( pcomps_ord**2, axis = 0 ) < 1.0001 ) ) ) )

np.sqrt( np.sum( pcomps_ord**2, axis = 0 ) )

In [ ]:
lda_ord.shape, exp_var.shape, pcomps_ord.shape, data.shape, data_stand.shape

### K-means Clustering

In [ ]:
K = data_stand.shape[1]
NDIMS_KMEANS = 2

In [ ]:
proj_data = [ pcomps_ord.T @ data_stand[:,i*num_cols:(i+1)*num_cols] for i in range( len( KEX ) ) ]
proj_data_red = [ _proj_data[ :NDIMS_KMEANS, : ] for _proj_data in proj_data ]

proj_data[0].shape, proj_data_red[0].shape, lda_ord[ :NDIMS_KMEANS ], exp_var[ :NDIMS_KMEANS ]

In [ ]:
np.sqrt(np.sum(data_stand**2, axis = 0))

In [ ]:
proj_data_red

In [ ]:
fig, ax = plt.subplots( 1, 1, figsize = ( 16, 10,) )

for i in range( len( KEX) ):
  ax.scatter( proj_data_red[i][ 0, :], proj_data_red[i][ 1, : ], label = KEX[i] )

ax.set_title( 'Ground Truth Cluster Plot (post-PCA to 2 Dimensions)' )
ax.set_xlabel( 'Principal Component 1' )
ax.set_ylabel( 'Principal Component 2' )
ax.legend()

fig.tight_layout()

fig.savefig( '../figures/ground-truth_cluster-plot_5mer.png' )

In [ ]:
# ==== K-MEANS CLUSTERING

final_means = np.stack( [ np.array( [ np.inf, np.inf ] ) for _ in range( len( KEX ) ) ], axis = 1 )
var = np.ones( ( len( KEX ), ), dtype = np.float64 ) * np.inf
final_all_classes = None
num_replace = 0
for _ in range( 10000 ):
  means = \
    np.random.rand( 2, len( KEX ) ) * \
    ( np.array( [ proj_data_red[i].max() - proj_data_red[i].min() for i in range( 2 ) ],
               dtype = np.float64 )[ :, np.newaxis ] ) + \
    ( np.array( [ proj_data_red[i].min() for i in range( 2 ) ], dtype = np.float64 )[ :, np.newaxis ] )
#   print( means_prev )
  proj_data_red_arr = np.concatenate( proj_data_red, axis = 1 )
  
  means_prev = np.stack( [ np.array( [ -np.inf, -np.inf ] ) for _ in range( len( KEX ) ) ], axis = 1 )
  while any( [ ( mp[0] != m[0] or mp[1] != m[1] ) for mp, m in zip( means_prev, means ) ] ):
    closest_class = np.fabs( np.sum( ( proj_data_red_arr[ :, :, np.newaxis ] - \
                                       means[ :, np.newaxis, : ] )**2, axis = 0 ) ).argmin( 1 )
    all_classes = np.unique( closest_class )
    assert all( [ c in range( len( KEX ) ) for c in all_classes ] )
#     print( all_classes )
    means_prev = means.copy()
    for c in all_classes:
      means[ :, c ] = np.mean( proj_data_red_arr[ :, closest_class == c ], axis = 1 )

  # Get variance
  closest_class = np.fabs( np.sum( ( proj_data_red_arr[ :, :, np.newaxis ] - \
                                     means[ :, np.newaxis, : ] )**2, axis = 0 ) ).argmin( 1 )
  all_classes = np.unique( closest_class )
  assert all( [ c in range( len( KEX ) ) for c in all_classes ] )
  old_var = var.copy()
  for c in all_classes:
    var[ c ] = np.sqrt(
      np.sum( ( np.std( proj_data_red_arr[ :, closest_class == c ], axis = 1 )**2 )**2, axis = 0 ) )

  # Get closest to ground truth
  # TODO: 

  if ( len( all_classes ) == len( KEX ) ):
  #   print( var )
    if np.sum( var ) < np.sum( old_var ):
      num_replace += 1
      final_all_classes = all_classes.copy()
      final_means = means.copy()

In [ ]:
assert ( len( final_all_classes ) == len( KEX ) )

In [ ]:
num_replace, var, old_var

In [ ]:
means_prev, means, final_means

In [ ]:
fig, ax = plt.subplots( 1, 1, figsize = ( 16, 10,) )

closest_class = np.fabs( np.sum( ( proj_data_red_arr[ :, :, np.newaxis ] - \
                                   final_means[ :, np.newaxis, : ] )**2, axis = 0 ) ).argmin( 1 )
all_classes = np.unique( closest_class )
assert all( [ c in range( len( KEX ) ) for c in all_classes ] )
for c in [ 1, 0, 2, 3, 4 ]:
  if c == 0:
    label = KEX[1]
  elif c == 1:
    label = KEX[0]
  else:
    label = KEX[c]
  ax.scatter( proj_data_red_arr[ 0, closest_class == c ],
              proj_data_red_arr[ 1, closest_class == c ], label = label )

ax.set_title( 'K-Means Cluster Plot (post-PCA to 2 Dimensions)' )
ax.set_xlabel( 'Principal Component 1' )
ax.set_ylabel( 'Principal Component 2' )
ax.legend()

fig.tight_layout()

fig.savefig( '../figures/k-means_cluster-plot_ladder.png' )